In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from parser import parse_config, data_converter, sample_set
from optimized_autoencoder import OptimizedAutoencoder


config_json = parse_config("resources/config.json")
activation_function = (lambda x: 1/(1+np.exp(-2*x)))
activation_derivative = (lambda x, *args: 2*(1/(1+np.exp(-2*x)))*(1-(1/(1+np.exp(-2*x)))) )
training_set = list()
testing_set = list()


#Parse and sample training data
for i in config_json["font_numbers"]:
    labels, alphabet = data_converter("resources/fonts_" + str(i) + ".txt")
    alphabet = np.array(alphabet)
    flattened_input = np.array(list(map(lambda char: np.array(char).flatten(), alphabet)))
    # if training_set is None:
    #     training_set = flattened_input
    # else:
    training_set.extend(sample_set(flattened_input, config_json["training_sample"]))
    testing_set.append(list(zip(labels, flattened_input)))


autoencoder = OptimizedAutoencoder(config_json["optimizer"],35, 2, config_json["architecture"], activation_function, activation_derivative)
autoencoder.train(training_set, training_set, config_json["epoch_limit"])



In [ ]:
n = 15
dimensions = (7, 5)
grid_x = np.linspace(0.05, 0.95, n)
grid_y = np.linspace(0.05, 0.95, n)
figure = np.zeros((dimensions[0] * n, dimensions[1] * n))
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        # Include bias in sample
        z_sample = np.array([xi, yi])
        x_decoded = autoencoder.decode(z_sample)
        x_decoded = np.array([0 if e > 0.5 else 1 for e in x_decoded])
        digit = x_decoded.reshape(dimensions[0], dimensions[1])
        figure[i * dimensions[0]: (i + 1) * dimensions[0],
            j * dimensions[1]: (j + 1) * dimensions[1]] = digit
plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()